In [1]:
!nvidia-smi

Fri Apr 18 07:35:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.86.15              Driver Version: 570.86.15      CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   54C    P8             13W /   72W |       1MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!apt update
!apt install libcudnn9-cuda-12
!pip install nvidia-cublas-cu12 nvidia-cudnn-cu12==9.*

!export LD_LIBRARY_PATH=`python3 -c 'import os; import nvidia.cublas.lib; import nvidia.cudnn.lib; print(os.path.dirname(nvidia.cublas.lib.__file__) + ":" + os.path.dirname(nvidia.cudnn.lib.__file__))'`


Get:1 https://packages.cloud.google.com/apt gcsfuse-jammy InRelease [1,227 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://packages.cloud.google.com/apt gcsfuse-jammy/main amd64 Packages [36.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [73.0 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,384 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,692 kB]
Get:13 http://archive.ubuntu.com/u

In [3]:
!pip3 install faster-whisper ctranslate2 datasets sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 13.4 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset, Dataset, Audio
import numpy as np

In [5]:
import os
cache_dir = "/content/huggingface_cache"
os.makedirs(cache_dir, exist_ok=True)

# Set ALL Hugging Face related cache directories
os.environ["TRANSFORMERS_CACHE"] = os.path.join(cache_dir, "transformers")
os.environ["HF_DATASETS_CACHE"] = os.path.join(cache_dir, "datasets")
os.environ["HF_HOME"] = os.path.join(cache_dir, "hf_home")
os.environ["HF_ASSETS_CACHE"] = os.path.join(cache_dir, "assets")
os.environ["HUGGINGFACE_HUB_CACHE"] = os.path.join(cache_dir, "hub")
os.environ["HF_MODULES_CACHE"] = os.path.join(cache_dir, "modules")

# Create all directories
for dir_path in [os.environ["TRANSFORMERS_CACHE"],
                os.environ["HF_DATASETS_CACHE"],
                os.environ["HF_HOME"],
                os.environ["HF_ASSETS_CACHE"],
                os.environ["HUGGINGFACE_HUB_CACHE"],
                os.environ["HF_MODULES_CACHE"]]:
    os.makedirs(dir_path, exist_ok=True)

# Force datasets to use the new cache
from datasets import config
config.HF_DATASETS_CACHE = os.environ["HF_DATASETS_CACHE"]

In [6]:
from huggingface_hub import notebook_login

notebook_login()

# Load the Dataset

In [7]:

dataset_name = "kreasof-ai/bem-audio-iwslt2025-test-set"
# test_dataset = load_dataset(dataset_name, trust_remote_code=True, split="test")
test_dataset = load_dataset(dataset_name, trust_remote_code=True, split="test")

README.md:   0%|          | 0.00/285 [00:00<?, ?B/s]

test-00000-of-00002.parquet:   0%|          | 0.00/329M [00:00<?, ?B/s]

test-00001-of-00002.parquet:   0%|          | 0.00/317M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/2779 [00:00<?, ? examples/s]

In [8]:
test_dataset

Dataset({
    features: ['audio'],
    num_rows: 2779
})

In [9]:
for sample in test_dataset:
    print(sample["audio"])  # Access the audio column
    break  # Just to check the first example

{'path': '15_7474_3_152_02_211030-180248_bem_8cc_elicit_1.wav', 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
       -9.15527344e-05, -6.10351562e-05, -6.10351562e-05]), 'sampling_rate': 16000}


# Create Arrays for audio

In [10]:
audio_arrays = [a["array"].astype(np.float32) for a in test_dataset["audio"]]

In [11]:
audio_arrays[:5]

[array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        -9.1552734e-05, -6.1035156e-05, -6.1035156e-05], dtype=float32),
 array([ 0.        ,  0.        ,  0.        , ..., -0.01599121,
        -0.01629639, -0.01654053], dtype=float32),
 array([ 0.0000000e+00, -3.0517578e-05, -3.0517578e-05, ...,
        -1.1291504e-03, -8.2397461e-04, -4.8828125e-04], dtype=float32),
 array([0.        , 0.        , 0.        , ..., 0.00061035, 0.00204468,
        0.00100708], dtype=float32),
 array([ 0.        ,  0.        ,  0.        , ...,  0.00094604,
         0.00024414, -0.00039673], dtype=float32)]

# Cascaded

## transcribe

In [42]:
model_id_asr = "kreasof-ai/whisper-medium-bem2en"
output_dir="ct2-whisper-small-transcription-finetuned"
commit_hash_asr = "46d8836114c5369ffee656a67a06e65ba68cb77d"

!ct2-transformers-converter \
--model {model_id_asr} \
--output_dir {output_dir} \
--revision {commit_hash_asr} \
--quantization float16 \
--copy_files tokenizer_config.json\
--force

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
config.json: 100% 1.38k/1.38k [00:00<00:00, 9.96MB/s]
2025-04-18 07:56:43.171756: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-18 07:56:43.191313: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-18 07:56:43.212379: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-18 07:56:43.2

In [43]:
from faster_whisper import WhisperModel

model_name = output_dir

model = WhisperModel(model_name, device="cuda", compute_type="float16")

In [44]:
test_dataset[0]

{'audio': {'path': '15_7474_3_152_02_211030-180248_bem_8cc_elicit_1.wav',
  'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         -9.15527344e-05, -6.10351562e-05, -6.10351562e-05]),
  'sampling_rate': 16000}}

In [45]:
tgt_lang = "en"

transcriptions = []
segments, info = model.transcribe(audio_arrays[0],
                                beam_size=5,
                                language=tgt_lang,
                                vad_filter=True
                                )
transcription = " ".join([segment.text.strip() for segment in segments])
transcriptions.append(transcription)

In [46]:
print(transcriptions)

['nafwala na amakalashi ku menso']


In [47]:
from tqdm.auto import tqdm

tgt_lang = "en"

transcriptions = []

for audio_array in tqdm(audio_arrays, total=len(audio_arrays)):
    segments, info = model.transcribe(audio_array,
                                        beam_size=5,
                                        language=tgt_lang,
                                        vad_filter=True)
    transcription = " ".join([segment.text.strip() for segment in segments])
    transcriptions.append(transcription)

  0%|          | 0/2779 [00:00<?, ?it/s]

In [48]:
print(*transcriptions[:20], sep="\n")

nafwala na amakalashi ku menso
Imbwa imo iyafonka pamona ilebutuka pa lunkoto lwa mucibansa
Akamwana nakemya ukulu mumulu ukulwisha ukutoba akapulanga.
Kunuma yabo kuli notuma motoka tulya babomfya mu ncende iyatalala nge iyi baliko.
abaume bali pamutenge ya ng'anda umo afwele ishati lya mitomito ilyamaboko ayatali elyo netoloshi lyamakumbi makumbi
Ifi bafweleko insapato fyakutelela nga baya mukwangala umu mwine muli ice.
Namayo aleenda mumusebo nabika nomwana pama beya.
Nimbona iyi incende balateyelako ifyangalo ifyapusana pusana emulandu wine kuli bambi abaletamba umupila wakuminwe elyo bambi baposele amano mutambi ba ifyangalo fimbi fye.
Namayo naikata ifyakulya pambale mukati ke tuuka.
Nangu limbi kuli bamo abamufulwishe.
Abantu bane bali umuli ifimabwe ifikulu nga nshi kabili nabafwala ne fimpompo ku mitwe yabo
Akamwana kambi balekafula amasapato kuli kafundisha wabo.
Afwile ale fwaya afike pa mpela ya mpili. Pantu ici icishimbi ekete eco ba bomfya aba temwa ukuniina impili
Kuli u

## MT (NLLB)

In [ ]:
mt_model_name = "kreasof-ai/nllb-200-3.3B-bem2en-flores200-bt"
output_mt = "ct2-nllb3.3b-be-en-finetuned"
commit_hash = "3dc4fe6449ec5ac06e45fcebc26e5221c6a8d7f5"
!ct2-transformers-converter --model {mt_model_name} --revision {commit_hash} --quantization float16 --output_dir {output_mt} --force

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-04-18 09:13:28.955297: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-18 09:13:29.403967: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-18 09:13:29.573788: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-18 09:13:29.624883: E external/local_xla/xla/stream_executor/cuda/c

In [59]:
# mt_model_name = "kreasof-ai/nllb-200-distilled-600M-bem2en-flores200"
# output_mt = "ct2-nllb-be-en-finetuned"
# commit_hash = "b7ab3b4345d080c83dd98b9c51d970d8d27dd18b"
# !ct2-transformers-converter --model {mt_model_name} --revision {commit_hash} --quantization float16 --output_dir {output_mt} --force

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-04-18 08:22:21.413930: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-18 08:22:21.431805: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-18 08:22:21.453639: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-18 08:22:21.460031: E external/local_xla/xla/stream_executor/cuda/c

In [60]:
# !wget https://s3.amazonaws.com/opennmt-models/nllb-200/flores200_sacrebleu_tokenizer_spm.model

In [78]:
import os

directory = "/content/"

ct2_model_name = output_mt

ct_model_path = os.path.join(directory, ct2_model_name)
sp_model_path = os.path.join(directory, "flores200_sacrebleu_tokenizer_spm.model")

In [79]:
import ctranslate2
import sentencepiece as spm
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

sp = spm.SentencePieceProcessor()
sp.load(sp_model_path)

translator = ctranslate2.Translator(ct_model_path, device=device)

RuntimeError: File model.bin is incomplete: failed to read a value of size 4 at position 0

In [ ]:
# Translate

# src_lang = "ben_Beng"
# tgt_lang = "eng_Latn"

src_lang = "bem_Latn"
tgt_lang = "eng_Latn"

beam_size = 5

source_sentences = transcriptions

source_sents = [sent.strip() for sent in source_sentences]
target_prefix = [[tgt_lang]] * len(source_sents)

# Subword the source sentences
source_sents_subworded = sp.encode_as_pieces(source_sents)
source_sents_subworded = [[src_lang] + sent + ["</s>"] for sent in source_sents_subworded]

# Translate the source sentences
translations = translator.translate_batch(source_sents_subworded,
                                          batch_type="tokens",
                                          max_batch_size=2024,
                                          beam_size=beam_size,
                                          target_prefix=target_prefix)
translations = [translation.hypotheses[0] for translation in translations]

# Desubword the target sentences
translations_desubword = sp.decode(translations)
translations_desubword = [sent[len(tgt_lang):].strip() for sent in translations_desubword]

print(*translations_desubword[:10], sep="\n")

In [ ]:
filename = f"{model_id_asr.split('/')[-1].replace('-','_')}_{commit_hash_asr[:7]}-{mt_model_name.split('/')[-1].replace('-','_')}_{commit_hash[:7]}"
filename

In [ ]:
filename_st_path = f"/content/output/{filename}-st.txt"
os.makedirs(os.path.dirname(filename_st_path), exist_ok=True)
with open(filename_st_path, "w", encoding="utf-8") as f:
    for sentence in translations_desubword:
        f.write(sentence + "\n")

In [ ]:
len(transcriptions)

In [ ]:
filename_asr = f"{model_id_asr.split('/')[-1].replace('-','_')}_{commit_hash_asr[:7]}"
filename_asr

In [ ]:
# filename_asr_path = f"/content/output/{filename_asr}-asr.txt"
# os.makedirs(os.path.dirname(filename_st_path), exist_ok=True)
# with open(filename_asr_path, "w", encoding="utf-8") as f:
#     for sentence in transcriptions:
#         f.write(sentence + "\n")